# IMPORT DATASETS

# *Spotify API - Import Data*

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [2]:
client_id = '42cd27984b66457a90a8bbafaaefc1c0'
client_secret = '32ec41e97f8e4d6b89f336e596bbd0ea'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
spotipy.Spotify(auth='TOKEN')

ids = []

In [3]:
def getTrackIDs(artist, track):
    try:
        results = sp.search(q='artist:' + str(artist) + ' track:' + str(track), type='track', limit = 1)
        if(len(results['tracks']['items']) != 0):
            trackId = results['tracks']['items'][0]['id']
            ids.append(trackId)
        
        else:
            ids.append('-')
        
    except KeyError as l:
        ids.append('-')

In [4]:
import time

links = pd.read_excel(r'musics\linkler.xlsx')
links.info()

for x in links.index:
    getTrackIDs(links['artist'][x],links['song_name'][x])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5102 entries, 0 to 5101
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        5102 non-null   datetime64[ns]
 1   artist      5102 non-null   object        
 2   song_name   5101 non-null   object        
 3   video       5102 non-null   object        
 4   group_type  5102 non-null   object        
 5   release     5089 non-null   object        
 6   link        5102 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 279.1+ KB


In [5]:
def getTrackFeatures(id):
    meta = sp.track(id)
    features = sp.audio_features(id)

    # meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']

  # features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']

    track = [name, album, artist, release_date, length, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
    return track

In [6]:
spotify = []

for x in ids:
    if x != '-':
        track = getTrackFeatures(x)
        spotify.append(track)
    else:
        track = ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-']
        spotify.append(track)

In [7]:
df = pd.DataFrame(spotify, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df.to_csv("musics\spotify_features.csv", sep = ',', index = False)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5102 entries, 0 to 5101
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              5102 non-null   object
 1   album             5102 non-null   object
 2   artist            5102 non-null   object
 3   release_date      5102 non-null   object
 4   length            5102 non-null   object
 5   popularity        5102 non-null   object
 6   danceability      5102 non-null   object
 7   acousticness      5102 non-null   object
 8   energy            5102 non-null   object
 9   instrumentalness  5102 non-null   object
 10  liveness          5102 non-null   object
 11  loudness          5102 non-null   object
 12  speechiness       5102 non-null   object
 13  tempo             5102 non-null   object
 14  time_signature    5102 non-null   object
dtypes: object(15)
memory usage: 598.0+ KB


# *YouTube API- Import Data*

In [10]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import urllib.parse as p
import re
import os
import pickle

SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [11]:
def youtube_authenticate():
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "client_secret_131652095285-ecm9map1hhbmnte3doqfe5tifupmv0hd.apps.googleusercontent.com.json"
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(client_secrets_file, SCOPES)
            creds = flow.run_local_server(port=0)
        # save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)

    return build(api_service_name, api_version, credentials=creds)

# authenticate to YouTube API
youtube = youtube_authenticate()

In [12]:
def get_video_id_by_url(url):
    """
    Return the Video ID from the video `url`
    """
    # split URL parts
    parsed_url = p.urlparse(url)
    # get the video ID by parsing the query of the URL
    video_id = p.parse_qs(parsed_url.query).get("v")
    if video_id:
        return video_id[0]
    else:
        raise Exception(f"Wasn't able to parse video URL: {url}")

In [13]:
def get_video_details(youtube, **kwargs):
    return youtube.videos().list(
        part="snippet,contentDetails,statistics",
        **kwargs
    ).execute()

In [14]:
def print_video_infos(video_response):
    try:
        if(len(video_response.get("items")) != 0):
            items = video_response.get("items")[0]
            statistics      = items["statistics"]
            like_count    = statistics["likeCount"]
            view_count    = statistics["viewCount"]
            like_counts.append(like_count)
            view_counts.append(view_count)
        else:
            like_counts.append(0)
            view_counts.append(0)
    except KeyError as l:
        like_counts.append(0)
        view_counts.append(0)

In [15]:
links = pd.read_excel(r'musics\linkler.xlsx')

like_counts = []
view_counts = []
count = 2
for x in links['link']:
    video_url = x
    count= count +1
    # parse video ID from URL
    video_id = get_video_id_by_url(video_url)
    # make API call to get video info
    response = get_video_details(youtube, id=video_id)
    # print extracted video infos
    print_video_infos(response)

In [16]:
links.insert(7, "like_count", like_counts, True)
links.insert(8, "view_count", view_counts, True)

In [17]:
links.to_csv('musics\videos_like_view_counts.csv', encoding='utf-8', index = False)
links.info()
links.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5102 entries, 0 to 5101
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        5102 non-null   datetime64[ns]
 1   artist      5102 non-null   object        
 2   song_name   5101 non-null   object        
 3   video       5102 non-null   object        
 4   group_type  5102 non-null   object        
 5   release     5089 non-null   object        
 6   link        5102 non-null   object        
 7   like_count  5102 non-null   object        
 8   view_count  5102 non-null   object        
dtypes: datetime64[ns](1), object(8)
memory usage: 358.9+ KB


,date,artist,song_name,video,group_type,release,link,like_count,view_count
0,2022-10-21,Mamamoo,"1,2,3 Eoi!",https://youtu.be/jpZF2zW38TE,Girl,Minor,https://www.youtube.com/watch?v=jpZF2zW38TE,143498,1565493
1,2022-10-21,ITZY,Boys Like You,https://youtu.be/6uZy86ePgO0,Girl,English,https://www.youtube.com/watch?v=6uZy86ePgO0,1074480,18656331
2,2022-10-20,Stray Kids,Chill,Youtube,Boy,Minor,https://www.youtube.com/watch?v=dtV1I9WtQuo,648317,7659405
3,2022-10-19,WEi,Spray,Youtube,Boy,Major,https://www.youtube.com/watch?v=oZQ-guD9BD8,54419,10681867
4,2022-10-18,Def.,my abandoned love,Youtube,Boy Solo,Minor,https://www.youtube.com/watch?v=BJa1N8YuHnY,38663,320851
5,2022-10-17,Stray Kids,SUPER BOARD,Youtube,Boy,Minor,https://www.youtube.com/watch?v=8ziUcTYwARI,801735,10718122
6,2022-10-17,N.Flying,I Like You,Youtube,Boy,Major,https://www.youtube.com/watch?v=YDSSk-YaZ0s,36483,805492
7,2022-10-17,(G)I-DLE,Nxde,Youtube,Girl,Major,https://www.youtube.com/watch?v=fCO7f0SmrDc,2676021,152608332
8,2022-10-17,Lee Chanhyuk,Panorama,Youtube,Boy Solo,Major,https://www.youtube.com/watch?v=Y7C8qIpo7Dg,95958,2377658
9,2022-10-16,LE SSERAFIM,ANTIFRAGILE,Youtube,Girl,Major,https://www.youtube.com/watch?v=pyf8cbqyfPs,2045627,93127126


# *Genius Lyrics- Import Lyrics*

In [2]:
import lyricsgenius as lg # https://github.com/johnwmillr/LyricsGenius
genius = lg.Genius('TISHgXKdwIY6lA79e2sAogJ2tBYA82vcj59yY5abRgwbwYcPJ1MUCCR7EMDWmdEy',  # Client access token from Genius Client API page
                             skip_non_songs=True, excluded_terms=["(Remix)", "(Live)"],
                             remove_section_headers=True)
genius.response_format = 'plain,html'

In [3]:
def get_lyrics(artists, song_limit):
    file = open("musics\lyrics_2.txt", "w", encoding="utf-8")  # File to write lyrics to
    c = 0
    for artist in artists:
        try:
            songs = (genius.search_artist(artist, max_songs=song_limit, sort='popularity')).songs
            song_lyrics_list = [song.lyrics for song in songs]
            file.write("\n \n \n \n \n".join(song_lyrics_list))
            c += 1
            print(f"Songs grabbed:{len(song_lyrics_list)}")
            
            print("___________________________________________________\n\n\n\n")
        except:
            print(f"some exception at {artist}: {c}")
    
    file.close()

    
def listToString(s):
 
    # initialize an empty string
    str1 = ""
 
    # traverse in the string
    for ele in s:
        str1 += ele
 
    # return string
    return str1

In [4]:
artist = pd.read_excel(r'musics\total_view_follower_of_groups.xlsx')
artist.info()

art = artist['group'].head(50)
lyrics = get_lyrics(art, 2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   group      204 non-null    object
 1   views      204 non-null    object
 2   followers  204 non-null    object
dtypes: object(3)
memory usage: 4.9+ KB
Searching for songs by  Blackpink ...

Changing artist name to 'BLACKPINK'
Song 1: "마지막처럼 (AS IF IT’S YOUR LAST)"
Song 2: "Lovesick Girls"

Reached user-specified song limit (2).
Done. Found 2 songs.
Songs grabbed:2
___________________________________________________




Searching for songs by  BTS...

Changing artist name to 'BTS'
Song 1: "Dynamite"
Song 2: "Butter"

Reached user-specified song limit (2).
Done. Found 2 songs.
Songs grabbed:2
___________________________________________________




Searching for songs by  Twice ...

Changing artist name to 'TWICE'
Song 1: "I CAN’T STOP ME (English Version)"
Song 2: "The Feels"

Reached user-specified son

Changing artist name to 'LE SSERAFIM'
Song 1: "FEARLESS"
Song 2: "ANTIFRAGILE"

Reached user-specified song limit (2).
Done. Found 2 songs.
Songs grabbed:2
___________________________________________________




Searching for songs by  PENTAGON ...

Changing artist name to 'PENTAGON'
Song 1: "빛나리 (Shine)"
Song 2: "데이지 (Daisy)"

Reached user-specified song limit (2).
Done. Found 2 songs.
Songs grabbed:2
___________________________________________________




Searching for songs by  Dreamcatcher ...

Changing artist name to 'DREAMCATCHER'
Song 1: "BEcause"
Song 2: "MAISON"

Reached user-specified song limit (2).
Done. Found 2 songs.
Songs grabbed:2
___________________________________________________




Searching for songs by  Nmixx ...

Changing artist name to 'NMIXX'
Song 1: "O.O"
Song 2: "占 (TANK)"

Reached user-specified song limit (2).
Done. Found 2 songs.
Songs grabbed:2
___________________________________________________




Searching for songs by  Secret Number ...

Changing arti